In [3]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt) # 실제로 계산해보면 값이 다르다

AIMessage(content='= $4923.38')

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(a, b):
    return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. This tool take two arguments, both  should be numbers till finish.",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
Thought: I can use the Sum Calculator tool to find the total cost of all the individual amounts provided.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 355.39, "b": 924.87}
}
```


Observation: 1280.26
Thought:I have calculated the sum of the first two amounts correctly. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.
Action: 
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 1280.26, "b": 721.2}
}
```
Observation: 2001.46
Thought:I have calculated the sum of the first three amounts correctly. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.

Action:
```
{
  "action": "Sum Calculator",
  "action_input": {"a": 2001.46, "b": 1940.29}
}
```
Observation: 3941.75
Thought:I have calculated the sum of the first four amounts correctly. Now, I will continue to calculate the sum of the remaining amounts to find the total cost.

Action:
```


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

In [3]:

from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(a, b):
    return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers.",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
I need to add up all these numbers to find the total cost.
Action: Sum Calculator
Action Input: 355.39 + 924.87

TypeError: plus() missing 1 required positional argument: 'b'

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)


def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\nExample:1,2",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...
I need to add up all the given numbers to find the total cost.
Action: Sum Calculator
Action Input: 355.39, 924.87
Observation: 1280.26
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 1280.26, 721.2
Observation: 2001.46
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 2001.46, 1940.29
Observation: 3941.75
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 3941.75, 573.63
Observation: 4515.38
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 4515.38, 65.72
Observation: 4581.1
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 4581.1, 35.00
Observation: 4616.1
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 46

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': '$5273.38'}

In [5]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        CalculatorTool(),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 355.39, 'b': 924.87}`


1280.26
Invoking: `CalculatorTool` with `{'a': 1280.26, 'b': 721.2}`


2001.46
Invoking: `CalculatorTool` with `{'a': 2001.46, 'b': 1940.29}`


3941.75
Invoking: `CalculatorTool` with `{'a': 3941.75, 'b': 573.63}`


4515.38
Invoking: `CalculatorTool` with `{'a': 4515.38, 'b': 65.72}`


4581.1
Invoking: `CalculatorTool` with `{'a': 4581.1, 'b': 35.0}`


4616.1
Invoking: `CalculatorTool` with `{'a': 4616.1, 'b': 552.0}`


5168.1
Invoking: `CalculatorTool` with `{'a': 5168.1, 'b': 76.16}`


5244.26
Invoking: `CalculatorTool` with `{'a': 5244.26, 'b': 29.12}`


5273.38The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

In [5]:
import requests
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

# You need to sign up for a free API key at https://www.alphavantage.co/
ALPHA_VANTAGE_API_KEY = "YOUR_API_KEY_HERE"

llm = ChatOpenAI(temperature=0.1)

class StockInfoToolArgsSchema(BaseModel):
    company: str = Field(
        description="The name of the company you want to get stock information for."
    )

class StockInfoTool(BaseTool):
    name = "StockInfoTool"
    description = """
    Use this tool to find stock market information for a company.
    It takes a company name as an argument and returns the stock symbol and basic information.
    """
    args_schema: Type[StockInfoToolArgsSchema] = StockInfoToolArgsSchema

    def _run(self, company: str):
        # First, search for the symbol
        search_url = f"https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={company}&apikey={ALPHA_VANTAGE_API_KEY}"
        search_response = requests.get(search_url)
        search_data = search_response.json()

        if "bestMatches" in search_data and search_data["bestMatches"]:
            symbol = search_data["bestMatches"][0]["1. symbol"]
            
            # Now, get the stock quote
            quote_url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}"
            quote_response = requests.get(quote_url)
            quote_data = quote_response.json()

            if "Global Quote" in quote_data:
                quote = quote_data["Global Quote"]
                return f"Stock Symbol: {symbol}\n" \
                       f"Price: ${quote.get('05. price', 'N/A')}\n" \
                       f"Change: {quote.get('09. change', 'N/A')} ({quote.get('10. change percent', 'N/A')})\n" \
                       f"Volume: {quote.get('06. volume', 'N/A')}\n"
            else:
                return f"Found symbol {symbol} for {company}, but couldn't retrieve quote information."
        else:
            return f"Couldn't find stock information for {company}."

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[StockInfoTool()],
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

try:
    result = agent.invoke(prompt)
    print(result)
except Exception as e:
    print(f"An error occurred: {e}")



> Entering new AgentExecutor chain...

Invoking: `StockInfoTool` with `{'company': 'Cloudflare'}`


Stock Symbol: NET
Price: $78.8600
Change: 0.3000 (0.3819%)
Volume: 1360307
The stock symbol for Cloudflare is NET. Currently, the stock is priced at $78.86 with a change of $0.30 (0.38%) and a trading volume of 1,360,307.

To analyze if Cloudflare is a potential good investment, we can consider the following factors:
1. **Company Performance**: Look at Cloudflare's financial reports, revenue growth, and profitability.
2. **Industry Trends**: Consider the growth potential of the cybersecurity industry, as Cloudflare operates in this sector.
3. **Competitive Position**: Evaluate Cloudflare's competitive advantage and market position compared to its peers.
4. **Future Outlook**: Assess any upcoming product launches, partnerships, or expansion plans that could impact the stock price.

It's important to conduct thorough research and consult with a financial advisor before making any investm

In [1]:
from langchain.schema import SystemMessage
import os
import requests
import textwrap

from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.document_loaders import WebBaseLoader
from langchain.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from bs4 import BeautifulSoup

llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-mini")

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class CustomWebBaseLoader(WebBaseLoader):
    def __init__(self, urls: list):
        self.urls = urls

    def scrape(self, url: str):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        if soup.header:
            soup.header.decompose()
        if soup.footer:
            soup.footer.decompose()

        return soup.get_text()

    def load(self):
        all_documents = []
        for url in self.urls:
            scraped_content = self.scrape(url)
            all_documents.append(scraped_content)
        return all_documents

class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Research about the XZ backdoor"
    )

class DuckDuckGoResultsArgsSchema(BaseModel):
    data: str = Field(
        description="The result of DuckDuckGoSearch."
    )

class DuckDuckGoUrlArgsSchema(BaseModel):
    urls: list = Field(
        description="List of urls you will scrape. Example urls: ['first url', 'second url', 'third url']"
    )



class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearch"
    description = """
    Use this tool to search by using DuckDuckGo.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchResults()
        return ddg.run(query)
        
class DuckDuckGoSearchGetUrlTool(BaseTool):
    name = "DuckDuckGoSearchGetUrl"
    description = """
    Use this tool to get the url used when searching by DuckDuckGo.
    It takes a string as an argument.
    """
    args_schema: Type[DuckDuckGoResultsArgsSchema] = DuckDuckGoResultsArgsSchema

    def _run(self, data):
        urls = []
        results_list = data.split("], [")
        for result in results_list:
            url = result.replace("[", "").replace("]", "").split(": ")[-1]
            urls.append(url)
            
        return urls

class DuckDuckGoSearchScrapeTool(BaseTool):
    name = "DuckDuckGoSearchScrape"
    description = """
    Use this tool to scrape the given websites used when searching by DuckDuckGo.
    It takes a list of urls as an argument.
    """
    args_schema: Type[DuckDuckGoUrlArgsSchema] = DuckDuckGoUrlArgsSchema

    def _run(self, urls):
        loader = CustomWebBaseLoader(urls)
        docs = loader.load()
        for index, doc in enumerate(docs):
            comment = f"\t----- Source: {urls[index]} -----\n\n\n"
            doc = '\n'.join(textwrap.wrap(doc, 70, break_long_words=False))
            save_file(doc, comment, "source")
        return "Finished Scraping and Saved file\n\n"

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearch"
    description = """
    Use this tool to search by using Wikipedia.
    It takes a query as an argument.
    """
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)
        


def save_file(doc, comment, file_name):
    file_path = f"./{file_name}.txt"
    if not os.path.exists(file_path):
        with open(file_path, 'w', encoding="utf-8") as f:
            f.write(comment)
            f.write(doc)
    else:
        with open(file_path, 'a', encoding="utf-8") as f:
            f.write(comment)
            f.write(doc)
            f.write("\n\n")

agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        DuckDuckGoSearchScrapeTool(),
        DuckDuckGoSearchGetUrlTool(),
    ],
    agent_kwargs={
        "system_message": SystemMessage(
            content="""
            You are a professional researcher.
            
            You search about the keyword given by using DuckDuckGo or Wikipedia.

            If DuckDuckGo is used to search about the keyword, scrape the url used and save in a file.

            If DuckDuckGoSearchTool is used, use DuckDuckGoSearchGetUrlTool to get the urls and then use DuckDuckGoSearchScrapeTool to scrape.

            After searching, explain about the keyword using the results of research.
        """
        )
    },
)

prompt = "Research about the XZ backdoor"

result = agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearch` with `{'query': 'XZ backdoor'}`


[snippet: What Does the Backdoor Do? Malicious code added to xz Utils versions 5.6.0 and 5.6.1 modified the way the software functions. The backdoor manipulated sshd, the executable file used to make remote ..., title: The XZ Backdoor: Everything You Need to Know - WIRED, link: https://www.wired.com/story/xz-backdoor-everything-you-need-to-know/], [snippet: That operation matches the style of the XZ Utils backdoor far more than the cruder supply chain attacks of APT41 or Lazarus, by comparison. "It could very well be someone else," says Aitel., title: The Mystery of 'Jia Tan,' the XZ Backdoor Mastermind - WIRED, link: https://www.wired.com/story/jia-tan-xz-backdoor/], [snippet: The xz-utils package, used by many Linux distributions, had a backdoor that could compromise sshd authentication. Learn how to check and update your Kali installation to avoid this issue., title: All about the